In [1]:
from csoundengine import *

sys <module 'sys' (built-in)>
builtins <module 'builtins' (built-in)>
_frozen_importlib <module '_frozen_importlib' (frozen)>
_imp <module '_imp' (built-in)>
_thread <module '_thread' (built-in)>
_warnings <module '_warnings' (built-in)>
_weakref <module '_weakref' (built-in)>
_io <module '_io' (built-in)>
marshal <module 'marshal' (built-in)>
posix <module 'posix' (built-in)>
_frozen_importlib_external <module '_frozen_importlib_external' (frozen)>
time <module 'time' (built-in)>
zipimport <module 'zipimport' (frozen)>
_codecs <module '_codecs' (built-in)>
codecs <module 'codecs' (frozen)>
encodings.aliases <module 'encodings.aliases' from '/usr/lib/python3.11/encodings/aliases.py'>
encodings <module 'encodings' from '/usr/lib/python3.11/encodings/__init__.py'>
encodings.utf_8 <module 'encodings.utf_8' from '/usr/lib/python3.11/encodings/utf_8.py'>
_signal <module '_signal' (built-in)>
_abc <module '_abc' (built-in)>
abc <module 'abc' (frozen)>
io <module 'io' (frozen)>
__main__ <modu

In [1]:
from csoundengine import *

e = Engine()
s = e.session()

{'sys': <module 'sys' (built-in)>, 'builtins': <module 'builtins' (built-in)>, '_frozen_importlib': <module '_frozen_importlib' (frozen)>, '_imp': <module '_imp' (built-in)>, '_thread': <module '_thread' (built-in)>, '_warnings': <module '_warnings' (built-in)>, '_weakref': <module '_weakref' (built-in)>, '_io': <module '_io' (built-in)>, 'marshal': <module 'marshal' (built-in)>, 'posix': <module 'posix' (built-in)>, '_frozen_importlib_external': <module '_frozen_importlib_external' (frozen)>, 'time': <module 'time' (built-in)>, 'zipimport': <module 'zipimport' (frozen)>, '_codecs': <module '_codecs' (built-in)>, 'codecs': <module 'codecs' (frozen)>, 'encodings.aliases': <module 'encodings.aliases' from '/usr/lib/python3.11/encodings/aliases.py'>, 'encodings': <module 'encodings' from '/usr/lib/python3.11/encodings/__init__.py'>, 'encodings.utf_8': <module 'encodings.utf_8' from '/usr/lib/python3.11/encodings/utf_8.py'>, '_signal': <module '_signal' (built-in)>, '_abc': <module '_abc' 



--Csound version 6.19 (double samples) Oct  1 2023
[commit: 593ac80dd986b58f24e3d59242243ba6266edda7]
libsndfile-1.2.0
sr = 48000.0, kr = 750.000, ksmps = 64
0dBFS level = 1.0, A4 tuning = 442.0
audio buffered in 1024 sample-frame blocks
reading 8192-byte blks of shorts from adc:Built-in Audio Pro: (RAW)
writing 2048 sample blks of 64-bit floats to dac:Built-in Audio Pro:
SECTION 1:


In [2]:
s.defInstr('test', r'''
ibus = p5
kfreq = busin:k(ibus)
outch 1, oscili:a(0.05, lag(kfreq, 0.01))
''')


Instr(test, ibus:5)

In [3]:
bus = s.assignBus(value=1000)
synth = s.sched('test', ibus=bus.token)
# e.writeBus(bus.token, 500)
bus.set(500, 0.001)
synth

Button(description='Stop', style=ButtonStyle())

Output()

Synth(▶ test:501.0002 start:0.513 dur:-1.000 p4=0 ibus:5=0)

In [10]:
bus.set(600)

here 0 0


In [18]:
buses = [e.assignBus(value=600) for _ in range(200)]
e.sync()
for bus in buses:
    synth = s.sched('test', dur=1, ibus=bus)
    e.writeBus(bus, 500, 0.001)
    

In [27]:
buses = [s.assignBus(value=600) for _ in range(200)]
s.engine.sync()
for bus in buses:
    synth = s.sched('test', dur=1, ibus=bus.token)
    bus.set(500, 0.002)
    

In [27]:
e.writeBus(bus.token, 1500)

In [28]:
bus.set(1600)

In [22]:
e.unschedAll()

end of section 1	 sect peak amps:  0.10000  0.00000
SECTION 1:


In [3]:
s.defInstr('sender', r'''
ibus = p5
ifreqbus = p6
kfreq = busin:k(ifreqbus)
asig vco2 0.1, kfreq
busout(ibus, asig)
''')

s.defInstr('receiver', r'''
ibus  = p5
kgain = p6
asig = busin:a(ibus)
asig *= a(kgain)
outch 1, asig
''')


Instr(receiver, ibus:5, kgain:6)

In [25]:
bus = s.assignBus()
freqbus = s.assignBus(value=600)

chain = SynthGroup([s.sched('sender', ibus=bus, ifreqbus=freqbus),
                    s.sched('receiver', ibus=bus, kgain=0.5)])
e.writeBus(freqbus.token, 400, delay=1)
chain

Button(description='Stop', style=ButtonStyle())

Output()

p1,start,dur,4,5:ibus,6:ifreqbus
501.0004 ▶,-0.013,-1.000,0,4,5
p1,start,dur,4,5:ibus,6:kgain
502.0004 ▶,-0.013,-1.000,0,4,0.5


In [27]:
e.writeBus(freqbus.token, 800)
freqbus.set(500)


--- direct access 5 2
--- direct access 5 2


In [29]:
freqbus.set(500)

--- direct access 5 2


In [31]:
e._dumpbus(freqbus.token)

Bus token=5, bus=2, kind=k, value=500.000000, refs=1


In [33]:
freqbus.release()

INIT ERROR in instr 32 (opcode initerror.s) line 691: 
   itoken -1 has no bus assigned to it

	initerror.s	#S1	
		   T 69.377 - note deleted.  i32 had 1 init errors


In [7]:
bus.release()

audio bus 0, token 0, refs before release 0
INIT ERROR in instr 32 (opcode initerror.s) line 679: 
   Bus pool is full!

	initerror.s	"Bus pool is full!"	
		   T 67.841 - note deleted.  i32 had 1 init errors


In [5]:
chain.stop()

audio bus 0, token 0, refs before release 3
scalar bus 0, token 1, refs before release 2
audio bus 0, token 0, refs before release 2


In [10]:
del freqbus

In [12]:
import gc
gc.collect()

0

In [4]:
nan = float('nan')
e.automateBus(freqbus, (0, nan, 2, 800, 2.1, 1000))

In [9]:
chain.stop()

In [7]:
chain.set(kfreq=1000)

KeyError: "Unknown parameter: 'kfreq'. Possible parameters for this event: {'kgain'}"

In [8]:
chain.stop()

In [10]:
chain.dynamicParams()

{'kgain'}